In [ ]:
# Mount to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Libraries
import pandas as pd

In [ ]:
# Reading the csv file
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/project/Trimester 3/RFLFSODataFull.csv')

In [ ]:
# @title Default title text
# Group by SYNOPCode and count the number of rows in each group
synop_group_data = data.groupby('SYNOPCode').size()

# Display the number of rows in each group
print(synop_group_data)

SYNOPCode
0    56964
3      191
4      466
5     6605
6    25018
7      419
8     1716
dtype: int64


Total Number of columns: 91379

weather conditions which has less than thousand data are kept, remainig weater conditions are down sized based on distance and time.
Take 120 data from approximately same distance and in that 120, take 5 data from each hour.

In [ ]:
data

,FSO_Att,RFL_Att,AbsoluteHumidity,AbsoluteHumidityMax,AbsoluteHumidityMin,Distance,Frequency,Particulate,ParticulateMax,ParticulateMin,...,TemperatureMax,TemperatureMin,Time,Visibility,VisibilityMax,VisibilityMin,WindDirection,WindSpeed,WindSpeedMax,WindSpeedMin
0,7.913289,6.927868,17.595709,17.615907,17.340148,2115.338398,83500000000,0.000000,0.000000,0.000000,...,26.603030,24.680849,9,65884.518460,72336.362233,65617.543754,88.755450,3.057066,6.863808,3.007939
1,7.451176,4.412096,17.549693,17.572415,17.299439,2113.999257,73500000000,0.000000,0.000000,0.000000,...,26.377164,24.313108,9,64963.410920,68753.386153,60379.327485,99.790057,2.727910,6.468903,2.537393
2,7.072747,6.268740,17.290230,17.644014,16.037894,2118.689047,83500000000,0.000000,0.000000,0.000000,...,27.670822,23.150277,10,54794.281070,54821.773817,50850.155963,65.730085,1.674810,2.826916,1.640809
3,6.949288,4.317853,16.820880,17.066776,15.895622,2114.632339,73500000000,0.000000,0.000000,0.000000,...,26.522100,23.174815,10,50205.641590,52519.927530,45374.510898,88.127497,0.962068,2.780643,0.886951
4,7.361052,6.114514,16.813820,17.953974,15.227225,2116.786055,83500000000,0.000000,0.000000,0.000000,...,26.305736,24.864100,10,59038.325990,64418.329138,54461.246506,84.167414,1.881007,4.476298,1.874052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91374,9.150012,11.968291,23.283256,24.142871,22.169805,3970.846129,83500000000,0.000000,0.000000,0.000000,...,27.257926,26.013205,4,13041.847800,13458.340663,12411.852994,91.391749,0.000000,0.027596,0.000000
91375,9.333777,11.538030,23.259618,25.104895,21.005496,3969.230652,73500000000,0.000000,0.000000,0.000000,...,28.101044,25.032282,4,15959.633590,16232.735439,15345.485232,47.997946,0.075640,0.349551,0.069834
91376,8.873168,11.491891,23.331697,23.996130,22.148649,3968.373512,83500000000,0.000000,0.000000,0.000000,...,28.249678,26.205773,4,13129.663750,13948.560177,11925.979953,103.963102,0.052282,0.178062,0.049824
91377,9.538886,11.526529,21.241027,21.624932,20.736391,3967.550165,73500000000,29.693160,30.761540,28.497137,...,26.880084,25.031934,5,15082.787860,15522.762221,14842.654291,82.090779,0.443520,1.838449,0.413146


In [ ]:

# Function to downsample data
# input the grouped SNOP which has more than 1000 data
# ouput is the downsampled data

def downsample_synop_data(group, target_size=120):
    # Round the 'Distance' to the nearest integer and group by the rounded distance

    distance_groups = group.groupby('Rounded_Distance')
    sampled_data = pd.DataFrame()

    # For each distance group, select a number of records
    for dist, dist_group in distance_groups:
        # Group by 'Time' within each distance group
        time_groups = dist_group.groupby('Time')

        # For each time group, randomly sample 5 records
        for _, time_group in time_groups:
            sampled_data = pd.concat([sampled_data, time_group.sample(min(5, len(time_group)))], ignore_index=True)

    # Return the downsampled group
    return sampled_data


downsampled_df = pd.DataFrame()
data['Rounded_Distance'] = data['Distance'].round()

# Apply downsampling based on the size of each SYNOPCode group
for code, group in data.groupby('SYNOPCode'):
    # Check if group size is small or large
    if len(group) <= 1000:  # Example threshold for small datasets
        downsampled_df = pd.concat([downsampled_df, group], ignore_index=True)  # Keep the entire group
    else:
        downsampled_group = downsample_synop_data(group)  # Apply downsampling to the larger group
        downsampled_df = pd.concat([downsampled_df, downsampled_group], ignore_index=True)


In [ ]:
# Group by SYNOPCode and count the number of rows in each group
synop_group = downsampled_df.groupby('SYNOPCode').size()

# Display the number of rows in each group
print(synop_group)

SYNOPCode
0    7707
3     191
4     466
5    3769
6    5078
7     419
8    1508
dtype: int64


Total no.of data after down sampling: 19138

eliminated no of data = 91379 - 19138 = 72241

In [ ]:
#  Drop a Rounded_Distance column
downsampled_df = downsampled_df.drop('Rounded_Distance', axis=1)

In [ ]:
file_path = '/content/drive/MyDrive/Colab Notebooks/project/Trimester 3/downsample_df.csv'
downsampled_df.to_csv(file_path, index=False)